In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

def train_cnn(csv_file):
    # Load the CSV file
    df = pd.read_csv(csv_file)
    
    # Filter only labels 0-35
    df = df[df.iloc[:, 0] <= 35]
    
    # Split into features and labels
    X = df.iloc[:, 1:].values / 255.0  # Normalize pixel values
    y = df.iloc[:, 0].values
    
    # Reshape to (N, 1, 28, 28) assuming 28x28 images
    X = X.reshape(-1, 1, 28, 28)
    
    # Convert to PyTorch tensors
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
    y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Define the CNN model
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
            self.bn1 = nn.BatchNorm2d(32)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(64)
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
            self.fc1 = nn.Linear(64 * 14 * 14, 128)
            self.fc2 = nn.Linear(128, 36)  # 36 output classes (0-35)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.5)
        
        def forward(self, x):
            x = self.relu(self.bn1(self.conv1(x)))
            x = self.pool(self.relu(self.bn2(self.conv2(x))))
            x = x.view(x.size(0), -1)  # Flatten
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.fc2(x)
            return x

    model = CNN()
    
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    epochs = 10
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")
    
    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    
    print(f'Test Accuracy: {correct / total:.4f}')
    
    return model

In [2]:
import cv2
import numpy as np
import os

# Create output directory
output_dir = "segmented_chars"
os.makedirs(output_dir, exist_ok=True)

# Load the license plate image
image = cv2.imread("licenseplate.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur and Edge Detection
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 150)

# Use morphological operations to enhance edges
kernel = np.ones((3, 3), np.uint8)
dilated = cv2.dilate(edges, kernel, iterations=1)

# Find contours
contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on size and aspect ratio
char_candidates = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = w / float(h)
    if 0.2 < aspect_ratio < 1.0 and 20 < w < 100 and 30 < h < 200:  # Adjust thresholds as needed
        char_candidates.append((x, y, w, h))

# Sort characters from left to right
char_candidates = sorted(char_candidates, key=lambda c: c[0])

# Extract and save segmented characters
for i, (x, y, w, h) in enumerate(char_candidates):
    char_img = gray[y:y+h, x:x+w]
    char_path = os.path.join(output_dir, f"char_{i}.png")
    cv2.imwrite(char_path, char_img)
    print(f"Saved: {char_path}")

# Save the processed image with bounding boxes
for (x, y, w, h) in char_candidates:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imwrite(os.path.join(output_dir, "segmented_plate.png"), image)
print(f"Saved segmented plate image with bounding boxes.")


Saved: segmented_chars\char_0.png
Saved: segmented_chars\char_1.png
Saved: segmented_chars\char_2.png
Saved: segmented_chars\char_3.png
Saved: segmented_chars\char_4.png
Saved: segmented_chars\char_5.png
Saved: segmented_chars\char_6.png
Saved: segmented_chars\char_7.png
Saved: segmented_chars\char_8.png
Saved: segmented_chars\char_9.png
Saved: segmented_chars\char_10.png
Saved: segmented_chars\char_11.png
Saved segmented plate image with bounding boxes.


In [ ]:
import cv2
import torch
import numpy as np
import os

# Load the trained model
model = train_cnn("typedCSV.csv")  # Or load a saved model using torch.load("model.pth")
model.eval()  # Set model to evaluation mode


Epoch 1/10, Loss: 1.8728
Epoch 2/10, Loss: 1.1510
Epoch 3/10, Loss: 0.9662
Epoch 4/10, Loss: 0.8475
Epoch 5/10, Loss: 0.7735
Epoch 6/10, Loss: 0.6961
Epoch 7/10, Loss: 0.6542
Epoch 8/10, Loss: 0.6118
Epoch 9/10, Loss: 0.5781
Epoch 10/10, Loss: 0.5535
Test Accuracy: 0.9572


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 784]

In [1]:

# Directory where segmented characters are saved
char_dir = "segmented_chars"
char_files = sorted(
    [f for f in os.listdir(char_dir) if f.startswith("char_") and f.endswith(".png")],
    key=lambda x: int(x.split("_")[1].split(".")[0])  # Extract numeric part and sort
)

# Define preprocessing function for CNN
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    img = cv2.resize(img, (28, 28))  # Resize to 28x28
    img = img.astype(np.float32) / 255.0  # Normalize to range [0,1]
    
    img = torch.tensor(img).unsqueeze(0).unsqueeze(0)  # Convert to tensor with shape (1, 1, 28, 28)
    return img

# Predict each segmented character
predictions = []
for file in char_files:
    img_path = os.path.join(char_dir, file)
    img_tensor = preprocess_image(img_path)  # No need to flatten
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_label = torch.max(output, 1)  # Get the predicted class
    
    predictions.append((file, predicted_label.item()))

# Function to map numerical labels to characters
def label_to_char(label):
    if 0 <= label <= 9:  # Numbers 0-9
        return str(label)
    elif 10 <= label <= 35:  # Uppercase A-Z
        return chr(ord('A') + (label - 10))
    elif 36 <= label <= 61:  # Lowercase a-z
        return chr(ord('a') + (label - 36))
    else:
        return '?'  # Unknown label

# Print results with actual characters
for filename, label in predictions:
    char = label_to_char(label)
    print(f"{filename}: Predicted Character = {char}")

NameError: name 'os' is not defined